In [5]:
import pandas as pd

df = pd.read_csv('~/Downloads/DEMO.csv', index_col=None, header=0)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['release_script'] = pd.to_datetime(df['release_script'])
bool(df.loc[0, 'timestamp'] < df.loc[0, 'release_script'])

True

In [3]:
df['date'] = [d.date() for d in df['timestamp']]
df['time'] = [d.time() for d in df['timestamp']]
df['price'] = df['close']

In [25]:
import numpy as np
def rolling_volatility(df, intvl=5):
    '''
    
    :param df: stock data frame, columns must have ['date', 'time', 'price'] 
    :param int: time interval, unit in minute
    :return: return a new data frame with its volatility
    '''
    
    data = df.copy()
    output = pd.DataFrame()
    
    for date in set(df.loc[:, 'date']):
        temp_df1 = data.loc[data.date == date, :]
        temp_df1['ind'] = temp_df1.index

        temp_df1 = temp_df1.groupby(temp_df1.index//intvl).mean()
        temp_df2 = temp_df1.set_index('ind')
        temp_df2.loc[:, 'time'] = df.loc[temp_df2.index, 'time']

        s_i = temp_df2['price']
        s_i_1 = temp_df2['price'].shift(1)
        temp_df2['u_sequence'] = np.log(s_i/s_i_1)
        s = temp_df2['u_sequence'].rolling(window=len(set(temp_df1.index//intvl)), center=False).std()
        #data.loc[data.date == date, str(intvl)+'_min_vol'] 
        temp_df2.loc[:, str(intvl)+'_min_vol'] = s * np.sqrt(len(set(temp_df1.index//intvl)))
        
        output = pd.concat([output,temp_df2])
    output.reset_index(drop=True,inplace=True)
    return output

def volatility(df, intvl=5):
    
    data = df.copy()
    avg_df = pd.DataFrame()
    output = pd.DataFrame(columns=['vol_before', 'vol_after'])
    
    for date in set(df.loc[:, 'date']):
        temp_df1 = data.loc[data.date == date, :]
        temp_df1['ind'] = temp_df1.index
        temp_df1 = temp_df1.groupby(temp_df1.index//intvl).mean()
        temp_df2 = temp_df1.set_index('ind')
        temp_df2.loc[:, 'timestamp'] = df.loc[temp_df2.index, 'timestamp']
        temp_df2.loc[:, 'release_script'] = df.loc[temp_df2.index, 'release_script']
        
        avg_df = pd.concat([avg_df,temp_df2])
    
    for date in set(avg_df.loc[:, 'release_script']):
        temp_df3 = avg_df.loc[avg_df.release_script == date,:]
        vol_before = temp_df3.loc[temp_df3.timestamp < temp_df3.release_script, 'price'].std()
        vol_after = temp_df3.loc[temp_df3.timestamp > temp_df3.release_script, 'price'].std()
        
        output.loc[date, :] = [vol_before, vol_after]
        output.index.name = 'date'
        output.sort_index(inplace=True)
    
    return output


def vol(df, mode):
    '''
    
    :param df: before or after stock data frame
    :param mode: 1 - before
                 0 - after
    :return: 
    '''
    
    intvl_list = [5,10,30,60,120]
    dic = {}
    
    for i in intvl_list:
        if mode:
            dic[str(i)+'_before'] = df.iloc[-i:, 7].std()
        else: 
            dic[str(i)+'_after'] = df.iloc[:i, 7].std()
    
    return dic


def auto_vol(df):
    
    data = df.copy()
    output = pd.DataFrame(columns=['10_after', '5_after', '120_before', '30_after', 
                                   '5_before', '60_before', '60_after', '10_before', '30_before', '120_after'])
        
    for date in set(data.loc[:, 'release_script']):
        
        temp_df1 = data.loc[data.release_script == date, :]
        
        df_before = temp_df1.loc[temp_df1.timestamp < temp_df1.release_script, :]
        
        df_after = temp_df1.loc[temp_df1.timestamp > temp_df1.release_script, :]
        
        dic ={**vol(df_before,1), **vol(df_after, 0)}
        
        for key in dic.keys():
            output.loc[date, key] = dic[key]
    
    return output
        

auto_vol(df)

{'10_after': 0.03427831352003497, '5_after': 0.005044799302250793, '120_before': 0.11615515883364574, '30_after': 0.05477905055137764, '5_before': 0.03033150177620696, '60_before': 0.13700669226716167, '60_after': 0.0688493917397813, '10_before': 0.042439499421071517, '30_before': 0.13088939350968554, '120_after': 0.08937666008422489}
{'10_after': 0.4219032023001381, '5_after': 0.28887713651308544, '120_before': 0.19597499443188413, '30_after': 0.30830924855918035, '5_before': 0.018708286933870735, '60_before': 0.13423501756500777, '60_after': 0.342944908107957, '10_before': 0.03167685625535194, '30_before': 0.07908256706628224, '120_after': 0.3015824738014488}
{'10_after': nan, '5_after': nan, '120_before': 0.0449869693791373, '30_after': nan, '5_before': 0.004086563348340505, '60_before': 0.046346266708907034, '60_after': nan, '10_before': 0.013615350813614454, '30_before': 0.014649334063558802, '120_after': nan}
{'10_after': nan, '5_after': nan, '120_before': 0.14055521609071675, '3

{'10_after': 0.21359944340335285, '5_after': 0.11606032913963357, '120_before': 0.3449150937302145, '30_after': 0.21948582547902, '5_before': 0.04073941580337161, '60_before': 0.3881408783609404, '60_after': 0.26125789583396936, '10_before': 0.08799122430990255, '30_before': 0.17210789184921665, '120_after': 0.22201156406141423}
{'10_after': nan, '5_after': nan, '120_before': 0.27831120690327077, '30_after': nan, '5_before': 0.033837848631377246, '60_before': 0.17720789702595538, '60_after': nan, '10_before': 0.08313476341993704, '30_before': 0.1694207599338163, '120_after': nan}
{'10_after': nan, '5_after': nan, '120_before': 0.2265493201291508, '30_after': nan, '5_before': 0.09249324299644901, '60_before': 0.18019079263772705, '60_after': nan, '10_before': 0.08111035007253288, '30_before': 0.08431195494783968, '120_after': nan}
{'10_after': 0.08112242873857027, '5_after': 0.1042362700790848, '120_before': 0.17498670933161198, '30_after': 0.07780796312998275, '5_before': 0.03898717737

,10_after,5_after,120_before,30_after,5_before,60_before,60_after,10_before,30_before,120_after
2013-04-25 10:00:00,0.0342783,0.0050448,0.116155,0.0547791,0.0303315,0.137007,0.0688494,0.0424395,0.130889,0.0893767
2013-02-12 09:00:00,0.421903,0.288877,0.195975,0.308309,0.0187083,0.134235,0.342945,0.0316769,0.0790826,0.301582
2013-10-30 10:00:00,NaN,NaN,0.044987,NaN,0.00408656,0.0463463,NaN,0.0136154,0.0146493,NaN
2015-10-28 10:00:00,NaN,NaN,0.140555,NaN,0.0194936,0.0462582,NaN,0.0252982,0.0468935,NaN
2014-02-05 10:00:00,NaN,NaN,0.125056,NaN,0.0696764,0.0760471,NaN,0.0633032,0.0470749,NaN
2015-02-04 10:00:00,NaN,NaN,0.154994,NaN,0.00547723,0.141269,NaN,0.0417532,0.106528,NaN
2016-04-28 10:00:00,NaN,NaN,0.0414631,NaN,0.0311448,0.0418386,NaN,0.0459627,0.0550119,NaN
2014-04-30 10:00:00,NaN,NaN,0.124477,NaN,0.0134164,0.0511988,NaN,0.0325278,0.059953,NaN
2012-07-25 09:00:00,0.345102,0.0221179,0.0623361,0.54051,0.0887012,0.0586099,0.553023,0.0655885,0.053236,0.615864
2017-02-08 17:30:00,NaN,NaN,0.0376711,NaN,0.0130384,0.0322445,NaN,0.0271633,0.0325105,NaN
